In [1]:
import dask
import dask.dataframe as ddf
import os
from glob import glob
import re
import pandas as pd
import numpy as np
import pickle

/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [2]:
user_group = pd.read_csv('result/all-data-editor-cluster-result.csv', sep="\t",encoding='utf-16')

In [3]:
path = '/home/ubuntu/scratch/xinrui/intermediate-result/all-data/sig-contributions-all-data/'

In [4]:
df = pd.read_parquet(path)

In [5]:
with open('../../all-data-article-clustering/articles-100-threshold.pkl' ,'rb') as f:
    selected_articles = pickle.load(f)

In [6]:
selected_talks = {'Talk:' + i for i in selected_articles}

In [7]:
selected_articles.update(selected_talks)

In [8]:
df = df[df['page.title'].isin(selected_articles)]

In [9]:
selected_editors = pd.read_parquet('/home/ubuntu/scratch/xinrui/mount-files/selected-editors-based-on-cc').index

In [10]:
df = df[df['contributor.username'].isin(selected_editors)]

In [11]:
user_contribution = df.groupby(['contributor.username','page.title']).count().rename(columns={'added_len':'sig.contributions'})

In [12]:
def top_ranked_articles(group):
    curr_user_group = user_group[user_group['group'] == group]
    all_users = curr_user_group['contributor.username'].unique()
    selected_df = user_contribution[user_contribution.index.get_level_values(0).isin(all_users)]
    res = {}
    for index in selected_df.index:
        if not index[1].lower().startswith('talk:'):
            if index[1] not in res:
                res[index[1]] = selected_df.loc[index]['sig.contributions']
            else:
                res[index[1]] += selected_df.loc[index]['sig.contributions']
    res = dict(sorted(res.items(), key=lambda item: item[1], reverse=True))
    return {k: res[k] for k in list(res)[:25]}

In [13]:
all_groups = user_group['group'].unique()

In [14]:
fin = {}

In [15]:
for group in all_groups:
    fin[group] = top_ranked_articles(group)

In [16]:
x = pd.DataFrame.from_dict(fin, orient="index").stack().to_frame()

In [17]:
x.reset_index(inplace=True)

In [18]:
x = x.rename(columns={'level_0':'group','level_1':'article',0:'num_sig_contributions'})

In [19]:
x = x.sort_values(['group', 'num_sig_contributions'],ascending=False)

In [24]:
x.to_csv('result/top-25-articles-for-each-group-by-sig-contributions.csv', index=False,sep="\t",encoding='utf-16')

In [25]:
# top 25 articles by revert

In [26]:
group_rv = pd.read_csv('result/group-revert-graph.csv', sep="\t",encoding='utf-16').set_index('Unnamed: 0')

In [29]:
revert_info = pd.read_parquet('result/all-data-revert-user-index-info-with-words-added-removed')

In [32]:
revert_info = revert_info.drop(columns={'added_words','removed_words'})

In [35]:
revert_info = revert_info.merge(user_group,left_on='user1',right_on='contributor.username').rename(columns={'group':'group1'})

In [36]:
revert_info = revert_info.merge(user_group,left_on='user2',right_on='contributor.username').rename(columns={'group':'group2'})

In [37]:
revert_to = revert_info.groupby(['group1','page.title']).agg({'group1':'count'}).rename(columns={'group1':'do_revert_to'})

In [38]:
revert_from = revert_info.groupby(['group2','page.title']).agg({'group1':'count'}).rename(columns={'group1':'receive_revert_from'})

In [39]:
revert_to.index.names = ['group','page.title']
revert_from.index.names = ['group','page.title']

In [41]:
res = pd.concat([revert_to.reset_index(),revert_from.reset_index()]).fillna(0)

In [42]:
res = res.groupby(['group','page.title']).agg({'do_revert_to':'sum','receive_revert_from':'sum'})

In [43]:
res['mutual_revert'] = res['do_revert_to'] + res['receive_revert_from']

In [51]:
fin = res.sort_values(['group','mutual_revert'],ascending=False).groupby(['group']).head(25)

In [52]:
fin.reset_index(inplace=True)

In [53]:
fin.to_csv('result/top-25-articles-for-each-group-by-mutual-revert.csv', index=False,sep="\t",encoding='utf-16')